In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from sklearn.metrics import classification_report, confusion_matrix, roc_curve, auc
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, LSTM, Dropout
import joblib

import warnings
warnings.filterwarnings("ignore")
import os

os.makedirs("models", exist_ok=True)
os.makedirs("scalers", exist_ok=True)


In [2]:
def create_sequences(data, seq_len=60):
    X, y = [], []
    for i in range(seq_len, len(data)):
        X.append(data[i-seq_len:i])
        y.append(data[i])
    return np.array(X), np.array(y)


In [4]:

stocks = ["CGH", "LICN", "NABIL", "NIFRA", "UPPER"]
all_results = []


In [5]:
for stock in stocks:
    print(f"\n=========== {stock} ===========")

    data = pd.read_csv(f"data/{stock}.csv")

    # ---------- Split ----------
    train_size = int(len(data)*0.7)
    val_size   = int(len(data)*0.15)

    train = data[:train_size]
    val   = data[train_size:train_size+val_size]
    test  = data[train_size+val_size:]

    # ---------- Scaling ----------
    scaler = MinMaxScaler()
    train_scaled = scaler.fit_transform(train[['close']])
    val_scaled   = scaler.transform(val[['close']])
    test_scaled  = scaler.transform(test[['close']])

    # ---------- Sequences ----------
    X_train, y_train = create_sequences(train_scaled)
    X_val, y_val     = create_sequences(val_scaled)
    X_test, y_test   = create_sequences(test_scaled)

    # ---------- LSTM ----------
    model = Sequential([
        LSTM(64, return_sequences=True, input_shape=(X_train.shape[1], 1)),
        Dropout(0.2),
        LSTM(64),
        Dropout(0.2),
        Dense(1)
    ])

    model.compile(optimizer="adam", loss="mse")

    model.fit(
        X_train, y_train,
        validation_data=(X_val, y_val),
        epochs=30,
        batch_size=32,
        verbose=1
    )

    # ✅ SAVE PROPERLY
    model.save(f"models/{stock}.keras")

    joblib.dump(scaler, f"scalers/{stock}_scaler.pkl")



=========== CGH ===========
Epoch 1/30
24/24 ━━━━━━━━━━━━━━━━━━━━ 5s 68ms/step - loss: 0.0338 - val_loss: 4.3425e-04
Epoch 2/30
24/24 ━━━━━━━━━━━━━━━━━━━━ 1s 48ms/step - loss: 0.0041 - val_loss: 0.0017
Epoch 3/30
24/24 ━━━━━━━━━━━━━━━━━━━━ 1s 49ms/step - loss: 0.0033 - val_loss: 0.0015
Epoch 4/30
24/24 ━━━━━━━━━━━━━━━━━━━━ 1s 48ms/step - loss: 0.0024 - val_loss: 1.4584e-04
Epoch 5/30
24/24 ━━━━━━━━━━━━━━━━━━━━ 1s 48ms/step - loss: 0.0023 - val_loss: 4.4100e-04
Epoch 6/30
24/24 ━━━━━━━━━━━━━━━━━━━━ 1s 48ms/step - loss: 0.0021 - val_loss: 1.1403e-04
Epoch 7/30
24/24 ━━━━━━━━━━━━━━━━━━━━ 1s 47ms/step - loss: 0.0019 - val_loss: 4.0972e-04
Epoch 8/30
24/24 ━━━━━━━━━━━━━━━━━━━━ 1s 48ms/step - loss: 0.0018 - val_loss: 7.6570e-05
Epoch 9/30
24/24 ━━━━━━━━━━━━━━━━━━━━ 1s 47ms/step - loss: 0.0022 - val_loss: 1.7860e-04
Epoch 10/30
24/24 ━━━━━━━━━━━━━━━━━━━━ 1s 49ms/step - loss: 0.0019 - val_loss: 7.2245e-05
Epoch 11/30
24/24 ━━━━━━━━━━━━━━━━━━━━ 1s 46ms/step - loss: 0.0021 - val_loss: 1.3490e-0

In [6]:
final_results = pd.DataFrame(all_results)
final_results

""
